## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import math

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,44.47,91,84,2.37,broken clouds
1,1,Busselton,AU,-33.6500,115.3333,54.45,56,94,9.01,overcast clouds
2,2,Yangjiang,CN,21.8500,111.9667,80.26,89,98,3.06,overcast clouds
3,3,Albany,US,42.6001,-73.9662,70.09,89,89,6.73,light rain
4,4,Avera,US,33.1940,-82.5271,94.91,58,56,13.51,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your destination? "))
max_temp = float(input("What is the maximum temperature you would like for your destination? "))

What is the minimum temperature you would like for your destination? 75
What is the maximum temperature you would like for your destination? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) &
                                       (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Yangjiang,CN,21.8500,111.9667,80.26,89,98,3.06,overcast clouds
5,5,Akyab,MM,20.1500,92.9000,79.61,87,100,2.93,overcast clouds
6,6,Kapaa,US,22.0752,-159.3190,84.52,78,90,5.01,light rain
11,11,Beyneu,KZ,45.3167,55.2000,87.33,15,3,9.31,clear sky
13,13,Katsuura,JP,35.1333,140.3000,84.31,76,4,5.30,clear sky
15,15,Ternate,ID,0.8000,127.4000,81.64,80,92,7.43,overcast clouds
17,17,Simbahan,PH,6.3032,120.5836,84.06,73,84,11.41,broken clouds
21,21,Uniontown,US,39.9001,-79.7164,75.54,49,67,1.01,light rain
28,28,Acarau,BR,-2.8856,-40.1200,78.62,75,85,14.09,overcast clouds
29,29,Jasper,US,37.2001,-94.3502,83.82,62,20,1.99,haze


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                249
City                   249
Country                249
Lat                    249
Lng                    249
Max Temp               249
Humidity               249
Cloudiness             249
Wind Speed             249
Current Description    249
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yangjiang,CN,80.26,overcast clouds,21.8500,111.9667,
5,Akyab,MM,79.61,overcast clouds,20.1500,92.9000,
6,Kapaa,US,84.52,light rain,22.0752,-159.3190,
11,Beyneu,KZ,87.33,clear sky,45.3167,55.2000,
13,Katsuura,JP,84.31,clear sky,35.1333,140.3000,
15,Ternate,ID,81.64,overcast clouds,0.8000,127.4000,
17,Simbahan,PH,84.06,broken clouds,6.3032,120.5836,
21,Uniontown,US,75.54,light rain,39.9001,-79.7164,
28,Acarau,BR,78.62,overcast clouds,-2.8856,-40.1200,
29,Jasper,US,83.82,haze,37.2001,-94.3502,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# Change empty string into Nan for Hotel Name Column.
hotel_nan = []
for hotel in hotel_df['Hotel Name'].values:
    if hotel != '':
        hotel_nan.append(hotel)
    else:
        hotel_nan.append(math.nan)
hotel_df['Hotel Name']= hotel_nan
hotel_df.head(12)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yangjiang,CN,80.26,overcast clouds,21.8500,111.9667,HUALUXE Yangjiang City Center
5,Akyab,MM,79.61,overcast clouds,20.1500,92.9000,Hotel Memory
6,Kapaa,US,84.52,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Beyneu,KZ,87.33,clear sky,45.3167,55.2000,NUR
13,Katsuura,JP,84.31,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
15,Ternate,ID,81.64,overcast clouds,0.8000,127.4000,Grand Dafam Bela Ternate
17,Simbahan,PH,84.06,broken clouds,6.3032,120.5836,NaN
21,Uniontown,US,75.54,light rain,39.9001,-79.7164,Fairfield Inn by Marriott Uniontown
28,Acarau,BR,78.62,overcast clouds,-2.8856,-40.1200,castelo pizzaria
29,Jasper,US,83.82,haze,37.2001,-94.3502,Guest House Motel


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Yangjiang,CN,80.26,overcast clouds,21.8500,111.9667,HUALUXE Yangjiang City Center
5,Akyab,MM,79.61,overcast clouds,20.1500,92.9000,Hotel Memory
6,Kapaa,US,84.52,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
11,Beyneu,KZ,87.33,clear sky,45.3167,55.2000,NUR
13,Katsuura,JP,84.31,clear sky,35.1333,140.3000,Katsuura Hotel Mikazuki
...,...,...,...,...,...,...,...
687,Tecoanapa,MX,86.58,broken clouds,16.5167,-98.7500,Hotel Manglares
690,Panorama,GR,81.52,clear sky,40.5878,23.0315,Panorama Hotel
691,Shaoxing,CN,84.15,light rain,30.0110,120.5715,Holiday Inn Shaoxing
694,Uribia,CO,88.29,clear sky,11.7139,-72.2660,Hotel Uribia


In [11]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30,31), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))